In [1]:

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model


In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices ((x_test, y_test)).batch(32)

In [0]:
class JacksModel(Model):
  def __init__ (self):
    super(JacksModel, self).__init__()
    self.conv1 = Conv2D(32,3,activation='tanh')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')
    
  def call(self,x):
    x=self.conv1(x)
    x=self.flatten(x)
    x=self.d1(x)
    return self.d2(x)

model = JacksModel()

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name= 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss=loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  
  train_loss(loss)
  train_accuracy(labels, predictions)

In [0]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)
  
  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [11]:
EPOCHS =5 

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)
  
  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)
    
   
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test loss: {}, Test accuracy: {} '
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))
  
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.19498524069786072, Accuracy: 94.14666748046875, Test loss: 0.11500980705022812, Test accuracy: 96.36000061035156 
Epoch 2, Loss: 0.08786483108997345, Accuracy: 97.2750015258789, Test loss: 0.09535275399684906, Test accuracy: 97.01000213623047 
Epoch 3, Loss: 0.057564131915569305, Accuracy: 98.20500183105469, Test loss: 0.10126838833093643, Test accuracy: 97.1199951171875 
Epoch 4, Loss: 0.041538357734680176, Accuracy: 98.58333587646484, Test loss: 0.10876462608575821, Test accuracy: 97.38999938964844 
Epoch 5, Loss: 0.03309961408376694, Accuracy: 98.89500427246094, Test loss: 0.15076234936714172, Test accuracy: 96.72000122070312 
